# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [1]:
import numpy as np
import requests as rq

# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(
        np.max(
            np.minimum(
                np.abs(query_vector-1),
                np.abs(query_vector+1)
            )
        ) == 0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Making random queries to the API

In [2]:
from scipy.optimize import linprog

In [3]:
challenge_id = 'testing' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

In [4]:
queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
query_results = query(challenge_id, queries)


In [5]:
print(queries.shape)
print(queries[0])

(200, 100)
[ 1  1 -1 -1 -1 -1  1 -1 -1 -1  1  1  1 -1 -1 -1 -1  1 -1 -1  1 -1 -1  1
 -1  1 -1  1 -1  1  1 -1  1 -1 -1  1 -1  1  1  1 -1 -1  1  1 -1 -1 -1  1
  1  1  1 -1 -1 -1 -1  1  1 -1  1 -1  1 -1 -1 -1  1 -1  1 -1  1  1  1  1
 -1 -1 -1  1 -1 -1 -1  1 -1 -1  1  1 -1 -1  1  1 -1  1 -1  1  1 -1  1  1
 -1 -1 -1  1]


In [6]:
print(query_results.shape)
print(query_results)

(200,)
[ 29.  13.  -8.   9. -19.   4.  -4.  -6.  -8. -12.  -9.  20.   8.  24.
  13.   2.   9.  17. -10.  -1. -16.   7. -28.  -8.   2.  22.  13.  30.
   5.   1.  11.  -0. -37.   5. -26. -10.  13.  10.  17.  19.   4.   5.
 -12.  29.  -4.   2.  25.  -3.   8.  14. -33. -28. -37. -16. -26.   4.
   8. -16. -22.  -2.  -3.  30.  -5.  33.   3.  13.  -0.   2.   3. -26.
 -29.   2.   9.  29.  17.   1. -22. -21.  19. -26.  -3.  11.  -2.  42.
 -12. -31.  -9. -14.   9.   4.  14.  -4. -16. -27.  27.  -1.  13. -26.
 -15.  27.  14. -45.   4.  -2.  45.  24.  27. -29. -10. -14.  -3. -11.
  -8. -15.  -3.  10. -11.  13.  17.   4.   2.  19. -11.   3.  -7.  -5.
   8.   6.  -3.  -4. -11.   2.  14. -23. -23.  -1.   4. -41. -16.   0.
  -2.  -7.  10. -15.   3.   9.  -8. -15. -19.  20. -15.  12. -46.  -1.
   3. -12.   9. -47.  -9.   4.  -8.   3.   4.  -0.   1.   3. -33.   6.
   8. -16.   2.  22.  -9.   9.  -4. -31.  34. -32.  -4.  13.  -7. -35.
  27.  -7.   7.  -5.  -5.   7.  -5.  36.   2.  -9.  21.  -3.   8.  15.

In [7]:
optimized_result = linprog(
    query_results @ queries,
    A_ub=queries,
    b_ub=np.abs(query_results),
    bounds=(-1,1)
)

In [8]:
optimized_result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -19688.69035206626
              x: [ 2.351e-01 -7.953e-01 ...  1.000e+00 -6.252e-01]
            nit: 73
          lower:  residual: [ 1.235e+00  2.047e-01 ...  2.000e+00
                              3.748e-01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [ 7.649e-01  1.795e+00 ...  0.000e+00
                              1.625e+00]
                 marginals: [ 0.000e+00  0.000e+00 ... -2.728e+02
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 3.038e+01  2.396e+01 ...  3.266e+01
                              1.438e+00]
                 marginals: [-0.000e+00 -0.000e+00 ... -0.000e+00
                             -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip

In [9]:
foo = -optimized_result.x

In [10]:
foo[foo >= 0] = 1
foo[foo < 0] = -1

In [11]:
foo.shape

(100,)

In [12]:
best_query_result = query("testing", foo, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")


Reconstruction attack achieves fraction 0.79 correct values


# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```



In [13]:
# best_query_number = np.argmax(query_results)
# best_query = queries[best_query_number]
# best_query_result = query(challenge_id, best_query, submit=True)
# print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")